In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit

class Evaluator:
    '''
    Don't change this code, it's not part of the assignment
    '''
    @staticmethod
    def generate_range(mu, sigma, weights, acc):
        rv1, rv2 = norm(loc=mu[0], scale=sigma[0]), norm(loc=mu[1], scale=sigma[1])
        start = min(rv1.ppf(acc), rv2.ppf(1-acc))
        end = max(rv1.ppf(acc), rv2.ppf(1-acc))
        return (start, end)
    
    @staticmethod
    def generate_x(mu_true, sigma_true, weights_true, mu_est, sigma_est, weights_est, num, acc):
        start_true, end_true = Evaluator.generate_range(mu_true, sigma_true, weights_true, acc)
        start_est, end_est = Evaluator.generate_range(mu_est, sigma_est, weights_est, acc)
        return np.linspace(min(start_true, start_est), max(end_true, end_est), num=num)
    
    @staticmethod
    def generate_y(x, mu, sigma, weights):
        rv1, rv2 = norm(loc=mu[0], scale=sigma[0]), norm(loc=mu[1], scale=sigma[1])
        cdf = weights[0] * rv1.cdf(x) + weights[1] * rv2.cdf(x)
        cdf = np.append(np.insert(cdf, 0, 0), 1)
        return np.diff(cdf, 1)
    
    @staticmethod
    def calculate_distance(mu_true, sigma_true, weights_true, mu_est, sigma_est, weights_est, num=1000, acc=0.001):      
        x = Evaluator.generate_x(mu_true, sigma_true, weights_true, mu_est, sigma_est, weights_est, num, acc)
        y_true = Evaluator.generate_y(x, mu_true, sigma_true, weights_true)
        y_est = Evaluator.generate_y(x, mu_est, sigma_est, weights_est)
        return np.abs(y_true - y_est).sum() / 2
    
class Sampler:
    '''
    Don't change this code, it's not part of the assignment
    '''
    
    @staticmethod
    def generate_observations(n, true_mu, true_sigma, true_weights):
        '''
        Generates list of n observations from two normal distributions: N(true_mu[0], true_sigma[0]) and 
        N(true_mu[1], true_sigma[1]). Each observation has probability true_weights[i] to 
        belong to N(true_mu[i], true_sigma[i]
        Args:
            n: integer, number of observations to generate
            true_mu: list of floats, length 2
            true_sigma: list of floats, length 2
            true_weights: list of floats, each element is between 0 and 1, sum(true_weights)==1
        Output:
            result: list of floats, length n, representing random draws from the 2 normal distributions as described.
            '''
        np.random.seed(42)
        n1 = np.random.binomial(n, true_weights[0])
        obs0 = np.random.normal(loc=true_mu[0], scale=true_sigma[0], size=n1).tolist()
        obs1 = np.random.normal(loc=true_mu[1], scale=true_sigma[1], size=n-n1).tolist()
        result = np.array(obs0 + obs1).reshape(-1, 1)
        np.random.shuffle(result)
        return result

class Problem:
        
    @staticmethod
    def estimate_parameters(obs):
        # Replace with your own code, but keep name of method equal
        
        def cdf_fun(obs, *guess):
            '''
            Calculates CDF of Gaussian mixture with weights weight_1 and weight_2
            '''
            mu1, mu2, std1, std2, weight_1 = guess
            weight_2 = 1 - weight_1
            cdf = weight_1 * norm.cdf(obs, loc=mu1, scale=std1) + weight_2 * norm.cdf(obs, loc=mu2, scale=std2)
            return cdf


        obs = np.sort(obs.flatten())
        cdf_obs = np.cumsum(np.ones(obs.shape))/len(obs) # empirical CDF

        # fit normal dist and use for initial guess
        mu0, s0 = norm.fit(obs)
        guess = [mu0, mu0, s0, s0, 0.5]
        
        param_bounds = ((mu0 - 4*s0, mu0 - 4*s0, 0, 0, 0), (mu0 + 4*s0, mu0 + 4*s0, 5*s0, 5*s0, 1))
        
        # fits cdf_fun to data
        params_est = curve_fit(cdf_fun,obs,cdf_obs,guess,bounds=param_bounds, method='trf')[0]

        mu1, mu2, std1, std2, weight_1 = params_est

        estimate_mu = [mu1, mu2]
        estimate_std = [std1, std2]
        estimate_weight = [weight_1, 1 - weight_1]

        return (estimate_mu, estimate_std, estimate_weight)

    
# Sample call and print, feel free to adjust    
n, true_mu, true_sigma, true_weights = 100, [-3, 5], [2, 1], [0.4, 0.6]

obs = Sampler.generate_observations(n, true_mu, true_sigma, true_weights)
est_mu, est_sigma, est_weights = Problem.estimate_parameters(obs)
print("Estimated parameters:")
print((est_mu, est_sigma, est_weights))
print("Calculated distance to true distribution:")
print(Evaluator.calculate_distance(true_mu, true_sigma, true_weights,
                                est_mu, est_sigma, est_weights, 1000))



n, true_mu, true_sigma, true_weights = 1000, [10, 1], [2, 2], [0.2, 0.8]

obs = Sampler.generate_observations(n, true_mu, true_sigma, true_weights)
est_mu, est_sigma, est_weights = Problem.estimate_parameters(obs)
print("Estimated parameters:")
print((est_mu, est_sigma, est_weights))
print("Calculated distance to true distribution:")
print(Evaluator.calculate_distance(true_mu, true_sigma, true_weights,
                                   est_mu, est_sigma, est_weights, 1000))

# Hard case - below optimizer fails to find a proper solution
n, true_mu, true_sigma, true_weights = 150, [1, 4], [1.5, 2.5], [0.2, 0.8]

obs = Sampler.generate_observations(n, true_mu, true_sigma, true_weights)
est_mu, est_sigma, est_weights = Problem.estimate_parameters(obs)
print("Estimated parameters:")
print((est_mu, est_sigma, est_weights))
print("Calculated distance to true distribution:")
print(Evaluator.calculate_distance(true_mu, true_sigma, true_weights,
                                   est_mu, est_sigma, est_weights, 1000))

Estimated parameters:
([4.945149207548788, -3.4611605928928646], [0.8372668879489034, 1.9984926508880823], [0.5916137913081562, 0.40838620869184383])
Calculated distance to true distribution:
0.08828146995745456
Estimated parameters:
([9.811383797905087, 1.0158363070779672], [1.9605208708337418, 1.9525313688011883], [0.1974847392282925, 0.8025152607717075])
Calculated distance to true distribution:
0.006199072579242015
Estimated parameters:
([6.635913526437677, 2.756458144977785], [0.9265426101717541, 2.3096288394420332], [0.16293880890595683, 0.8370611910940432])
Calculated distance to true distribution:
0.058789808712393476
